In [1]:
import pandas as pd
import random
import numpy as np
from scipy.spatial import distance_matrix
import itertools

In [2]:
def generate_dm(df, show=True):
    temp = df[[0, 1]].to_numpy()
    dm = distance_matrix(temp, temp)

    temp = df[2].to_numpy() // 2
    temp = temp * np.ones((200, 200))
    dm = dm + temp + temp.T
    dm = dm // 1

    for i in range(200):
        dm[i][i] = np.inf

    if show:
        df_dist = pd.DataFrame(dm)
        # display(df_dist)
    return dm


def get_random_solution():
    return random.sample([i for i in range(0, 200)], 100)


def calulate_total_cost(path, dm):
    total = 0
    nr = len(path)
    for idx, node in enumerate(path):
        total += dm[node][path[(idx + 1) % nr]]
    return total

In [98]:
def generate_all_moves(path, LM, dm):
    new_nodes = list(set([i for i in range(0, 200)]) - set(path))
    
    #inter
    for idx, node in enumerate(path):
        for new_node in new_nodes:
            new_cost = dm[path[idx-1]][new_node] + dm[path[(idx+1)%100]][new_node]
            new_cost -= (dm[path[idx-1]][node] + dm[path[(idx+1)%100]][node])
            if new_cost < 0:
                old_move = [path[idx-1], path[idx], path[(idx+1)%100]]
                new_move = [path[idx-1], new_node, path[(idx+1)%100]]
                LM.append([new_cost, [old_move, new_move]])
    #intra
    for idx1, idx2 in all_posibble_comb:
        edge1, edge2 = [path[idx1], path[(idx1+1)%100]], [path[idx2], path[(idx2+1)%100]]
        if not(edge1[0] in edge2 or edge1[1] in edge2):
            new_cost = dm[edge1[0]][edge2[0]] + dm[edge1[1]][edge2[1]] 
            new_cost -= (dm[edge1[0]][edge1[1]] + dm[edge2[0]][edge2[1]])
            if new_cost < 0:
                LM.append([new_cost, [edge1, edge2]])
        
        
    
    
    return LM

In [122]:
def check_move(path, move):
    if len(move[0]) == 3:
        if move[0][1] in path and move[1][1] not in path: #if old nodes are in same order in path, and new node not in path
            pos0 = path.index(move[0][1])
            if path[pos0-1] == move[0][0] and path[(pos0+1)%100] == move[0][2]:
                return True, True
            if path[(pos0+1)%100] == move[0][0] and path[pos0-1] == move[0][2]:
                return True, True 
        return False, False
    else:
        edge1 = move[0]
        edge2 = move[1]

        if edge1[0] in path and edge1[1] in path and edge2[0] in path and edge2[1] in path:
            n1, n2, x1, x2 = path.index(edge1[0]), path.index(edge1[1]), path.index(edge2[0]), path.index(edge2[1])
#             print(n1, n2, x1, x2)
            if (n2-n1 == 1 and x2-x1 == 1) or (n2-n1 == -1 and x2-x1 == -1):
                return True, True
            if (n2-n1 == -1 and x2-x1 == 1) or (n2-n1 == 1 and x2-x1 == -1):
                return False, True
            
        return False, False


In [123]:
def new_moves_node(path, change_node_index, LM):
    new_nodes = list(set([i for i in range(0, 200)]) - set(path))
    for new_node in new_nodes:
        new_cost = dm[path[change_node_index-1]][new_node] + dm[path[(change_node_index+1)%100]][new_node]
        new_cost -= (dm[path[change_node_index-1]][path[change_node_index]] + dm[path[(change_node_index+1)%100]][path[change_node_index]])
#         print("new cost", new_cost)
        if new_cost < 0:
            old_move = [path[change_node_index-1], path[change_node_index], path[(change_node_index+1)%100]]
            new_move = [path[change_node_index-1], new_node, path[(change_node_index+1)%100]]
            LM.append([new_cost, [old_move, new_move]])
    return LM

In [124]:
def new_moves_edge_exchange(path, change_node_index, LM):
    for idx in range(len(path)):
        if idx != change_node_index:
            edge1, edge2 = [path[idx], path[(idx+1)%100]], [path[change_node_index], path[(change_node_index+1)%100]]
            if not(edge1[0] in edge2 or edge1[1] in edge2):
                new_cost = dm[edge1[0]][edge2[0]] + dm[edge1[1]][edge2[1]] 
                new_cost -= (dm[edge1[0]][edge1[1]] + dm[edge2[0]][edge2[1]])
                if new_cost < 0:
                    LM.append([new_cost, [edge1, edge2]])
    return LM
#     pass


In [128]:
all_posibble_comb = list(itertools.combinations([i for i in range(100)], 2))
tsp = pd.read_csv('TSPC.csv', sep=';', header=None)
dm = generate_dm(tsp)
path = get_random_solution()

LM = []
LM = generate_all_moves(path, [], dm)
print(calulate_total_cost(path, dm))
change = True

while change:

    to_delete = []
    change = False
#     print('LM', len(LM))
    LM.sort(key=lambda x: x[0])
    if cost_move[0] > 0:
        break
        
    for cost_move in LM:
        
        cost, move = cost_move
        good, keep = check_move(path, move)
        if good:
#             print(move, cost)
            if len(move[0]) == 3:
                
                idx = path.index(move[0][1])
                path[idx] = move[1][1]
                change = True
#                 print('old LM', len(LM))
                LM = new_moves_edge_exchange(path, idx, LM)
#                 print('new LM', len(LM))
                to_delete.append(cost_move)
#                 print(calulate_total_cost(path, dm))   
                break
            else:
                edge1, edge2 = move
                n1, n2, x1, x2 = path.index(edge1[0]), path.index(edge1[1]), path.index(edge2[0]), path.index(edge2[1])
                path[n1: x1+1] = path[n1: x1+1][::-1]
                change = True
                LM = new_moves_node(path, n1, LM)
                LM = new_moves_node(path, x1, LM)

#                 print(calulate_total_cost(path, dm))
                to_delete.append(cost_move)
                break

                
        if not keep:
            to_delete.append(cost_move)

    for cost_move in to_delete:
        LM.remove(cost_move)
    # instead removing one by one, remove whole by indexes
    
print(calulate_total_cost(path, dm))            


199738.0
67214.0


In [59]:

import copy
path2 = copy.deepcopy(path)
path2[idx1:idx2+1] = path2[idx1:idx2+1][::-1]
path == path2

False

In [61]:
idx1, idx2

(34, 53)

In [12]:
pairs = list(
            itertools.product(np.arange(int(np.ceil(200 * 0.5))), np.arange(int(np.ceil(200 * 0.5)) + 1)))

In [79]:
a = [0, 1, 2, 3, 4, 5, 6,7]
a[2: 5+1] = a[2: 5+1][::-1]
a

[0, 1, 5, 4, 3, 2, 6, 7]

In [16]:
sum([i for i in range(200)])

19900

In [21]:
all_posibble_comb = list(itertools.combinations([i for i in range(200)], 2))
all_posibble_comb

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 20),
 (0, 21),
 (0, 22),
 (0, 23),
 (0, 24),
 (0, 25),
 (0, 26),
 (0, 27),
 (0, 28),
 (0, 29),
 (0, 30),
 (0, 31),
 (0, 32),
 (0, 33),
 (0, 34),
 (0, 35),
 (0, 36),
 (0, 37),
 (0, 38),
 (0, 39),
 (0, 40),
 (0, 41),
 (0, 42),
 (0, 43),
 (0, 44),
 (0, 45),
 (0, 46),
 (0, 47),
 (0, 48),
 (0, 49),
 (0, 50),
 (0, 51),
 (0, 52),
 (0, 53),
 (0, 54),
 (0, 55),
 (0, 56),
 (0, 57),
 (0, 58),
 (0, 59),
 (0, 60),
 (0, 61),
 (0, 62),
 (0, 63),
 (0, 64),
 (0, 65),
 (0, 66),
 (0, 67),
 (0, 68),
 (0, 69),
 (0, 70),
 (0, 71),
 (0, 72),
 (0, 73),
 (0, 74),
 (0, 75),
 (0, 76),
 (0, 77),
 (0, 78),
 (0, 79),
 (0, 80),
 (0, 81),
 (0, 82),
 (0, 83),
 (0, 84),
 (0, 85),
 (0, 86),
 (0, 87),
 (0, 88),
 (0, 89),
 (0, 90),
 (0, 91),
 (0, 92),
 (0, 93),
 (0, 94),
 (0, 95),
 (0, 96),
 (0, 97),
 (0, 98),
 (0, 99),
 (0, 100),
 (0, 101